<a href="https://colab.research.google.com/github/lakecg2/practica_progavan/blob/main/cafeteria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from enum import Enum
from typing import List, Dict
from datetime import datetime
from dataclasses import dataclass, field
import os

class RolEmpleado(Enum):
    MESERO = "mesero"
    BARISTA = "barista"
    GERENTE = "gerente"

class EstadoPedido(Enum):
    PENDIENTE = "pendiente"
    EN_PREPARACION = "en_preparacion"
    ENTREGADO = "entregado"

class TipoBebida(Enum):
    CALIENTE = "caliente"
    FRIA = "fria"

@dataclass
class Persona:
    nombre: str
    id: str
    telefono: str

class Cliente(Persona):
    def __init__(self, nombre: str, id: str, telefono: str):
        super().__init__(nombre, id, telefono)
        self.historial_pedidos: List['Pedido'] = []
        self.puntos_fidelidad: int = 0

    def realizar_pedido(self, productos: List['ProductoPersonalizado'], cafeteria: 'Cafeteria') -> 'Pedido':
        pedido = Pedido(self, productos)
        if cafeteria.validar_inventario(pedido):
            self.historial_pedidos.append(pedido)
            cafeteria.agregar_pedido(pedido)
            self.puntos_fidelidad += len(productos)
            return pedido
        raise ValueError("No hay suficientes ingredientes para completar el pedido")

    def consultar_historial(self) -> None:
        print(f"\nHistorial de pedidos de {self.nombre}:")
        for pedido in self.historial_pedidos:
            print(f"\nPedido del {pedido.fecha_creacion.strftime('%d/%m/%Y %H:%M')}")
            print(f"Estado: {pedido.estado.value}")
            print("Productos:")
            for producto in pedido.productos:
                print(f"- {producto.nombre}: ${producto.precio_final:.2f}")
            print(f"Total: ${pedido.total:.2f}")

class Empleado(Persona):
    def __init__(self, nombre: str, id: str, telefono: str, rol: RolEmpleado):
        super().__init__(nombre, id, telefono)
        self.rol = rol

    def actualizar_inventario(self, inventario: 'Inventario', ingrediente: str, cantidad: int) -> None:
        if self.rol != RolEmpleado.GERENTE and self.rol != RolEmpleado.BARISTA:
            raise PermissionError("No tienes permiso para actualizar el inventario")
        inventario.actualizar_cantidad(ingrediente, cantidad)

    def actualizar_estado_pedido(self, pedido: 'Pedido', nuevo_estado: EstadoPedido) -> None:
        pedido.actualizar_estado(nuevo_estado)
        print(f"\nPedido actualizado a: {nuevo_estado.value}")

@dataclass
class ProductoBase:
    nombre: str
    precio_base: float
    descripcion: str

class Bebida(ProductoBase):
    def __init__(self, nombre: str, precio_base: float, descripcion: str, tipo: TipoBebida):
        super().__init__(nombre, precio_base, descripcion)
        self.tipo = tipo
        self.ingredientes_base: Dict[str, int] = {}

    def agregar_ingrediente_base(self, ingrediente: str, cantidad: int) -> None:
        self.ingredientes_base[ingrediente] = cantidad

@dataclass
class Postre(ProductoBase):
    es_vegano: bool = False
    sin_gluten: bool = False
    ingredientes_base: Dict[str, int] = field(default_factory=dict)

class ProductoPersonalizado:
    def __init__(self, producto_base: ProductoBase):
        self.producto_base = producto_base
        self.modificaciones: Dict[str, int] = {}
        self.precio_final = producto_base.precio_base

    def agregar_modificacion(self, ingrediente: str, cantidad: int) -> None:
        self.modificaciones[ingrediente] = cantidad
        if cantidad > 0:  # Si es extra ingrediente
            self.precio_final += 0.5  # Cargo extra por modificación

    @property
    def nombre(self) -> str:
        nombre = self.producto_base.nombre
        if self.modificaciones:
            modificaciones = ", ".join(f"{'extra' if cant > 0 else 'sin'} {ing}"
                                     for ing, cant in self.modificaciones.items())
            nombre += f" ({modificaciones})"
        return nombre

class Inventario:
    def __init__(self):
        self.stock: Dict[str, int] = {}

    def agregar_ingrediente(self, ingrediente: str, cantidad: int) -> None:
        self.stock[ingrediente] = cantidad

    def actualizar_cantidad(self, ingrediente: str, cantidad: int) -> None:
        if ingrediente not in self.stock:
            raise ValueError(f"El ingrediente {ingrediente} no existe en el inventario")
        nuevo_stock = self.stock[ingrediente] + cantidad
        if nuevo_stock < 0:
            raise ValueError("No puede haber stock negativo")
        self.stock[ingrediente] = nuevo_stock

    def hay_suficiente_stock(self, ingrediente: str, cantidad_requerida: int) -> bool:
        return self.stock.get(ingrediente, 0) >= cantidad_requerida

    def mostrar_stock(self) -> None:
        print("\nInventario actual:")
        for ingrediente, cantidad in self.stock.items():
            print(f"{ingrediente}: {cantidad} unidades")

class Pedido:
    def __init__(self, cliente: Cliente, productos: List[ProductoPersonalizado]):
        self.cliente = cliente
        self.productos = productos
        self.estado = EstadoPedido.PENDIENTE
        self.fecha_creacion = datetime.now()
        self.promociones_aplicadas: List[Promocion] = []
        self._calcular_total()

    def _calcular_total(self) -> None:
        subtotal = sum(producto.precio_final for producto in self.productos)
        self.total = subtotal
        for promocion in self.promociones_aplicadas:
            self.total = promocion.aplicar_descuento(self.total)

    def actualizar_estado(self, nuevo_estado: EstadoPedido) -> None:
        self.estado = nuevo_estado

    def aplicar_promocion(self, promocion: 'Promocion') -> None:
        if promocion.es_aplicable(self):
            self.promociones_aplicadas.append(promocion)
            self._calcular_total()

class Promocion:
    def __init__(self, nombre: str, descuento_porcentaje: float,
                 puntos_requeridos: int = 0, productos_minimos: int = 0):
        self.nombre = nombre
        self.descuento_porcentaje = descuento_porcentaje
        self.puntos_requeridos = puntos_requeridos
        self.productos_minimos = productos_minimos

    def es_aplicable(self, pedido: Pedido) -> bool:
        if self.puntos_requeridos > 0:
            if pedido.cliente.puntos_fidelidad < self.puntos_requeridos:
                return False
        if self.productos_minimos > 0:
            if len(pedido.productos) < self.productos_minimos:
                return False
        return True

    def aplicar_descuento(self, total: float) -> float:
        return total * (1 - self.descuento_porcentaje / 100)

class Cafeteria:
    def __init__(self):
        self.inventario = Inventario()
        self.pedidos: List[Pedido] = []
        self.menu: Dict[str, ProductoBase] = {}
        self.promociones: List[Promocion] = []

    def agregar_al_menu(self, producto: ProductoBase) -> None:
        self.menu[producto.nombre] = producto

    def mostrar_menu(self) -> None:
        print("\nMenú de la Cafetería:")
        print("\nBEBIDAS:")
        for producto in self.menu.values():
            if isinstance(producto, Bebida):
                print(f"{producto.nombre}: ${producto.precio_base:.2f}")
                print(f"  {producto.descripcion}")

        print("\nPOSTRES:")
        for producto in self.menu.values():
            if isinstance(producto, Postre):
                print(f"{producto.nombre}: ${producto.precio_base:.2f}")
                print(f"  {producto.descripcion}")
                if producto.es_vegano:
                    print("  (Vegano)")
                if producto.sin_gluten:
                    print("  (Sin Gluten)")

    def agregar_pedido(self, pedido: Pedido) -> None:
        self.pedidos.append(pedido)
        self._actualizar_inventario_por_pedido(pedido)

    def _actualizar_inventario_por_pedido(self, pedido: Pedido) -> None:
        for producto in pedido.productos:
            if isinstance(producto.producto_base, (Bebida, Postre)):
                for ing, cant in producto.producto_base.ingredientes_base.items():
                    self.inventario.actualizar_cantidad(ing, -cant)
            for ing, cant in producto.modificaciones.items():
                self.inventario.actualizar_cantidad(ing, -cant)

    def validar_inventario(self, pedido: Pedido) -> bool:
        for producto in pedido.productos:
            if isinstance(producto.producto_base, (Bebida, Postre)):
                for ing, cant in producto.producto_base.ingredientes_base.items():
                    if not self.inventario.hay_suficiente_stock(ing, cant):
                        return False
            for ing, cant in producto.modificaciones.items():
                if not self.inventario.hay_suficiente_stock(ing, cant):
                    return False
        return True

def inicializar_cafeteria() -> Cafeteria:
    cafe = Cafeteria()

    # Configurar inventario inicial
    ingredientes = {
        "cafe": 1000,
        "leche": 2000,
        "leche_almendra": 1000,
        "azucar": 1500,
        "chocolate": 1000,
        "te_negro": 500,
        "te_verde": 500,
        "crema": 1000,
        "caramelo": 500,
        "vainilla": 500,
        "harina": 2000,
        "huevos": 100,
        "frutas": 500
    }

    for ingrediente, cantidad in ingredientes.items():
        cafe.inventario.agregar_ingrediente(ingrediente, cantidad)

    # Crear bebidas
    bebidas = [
        ("Café Americano", 2.50, "Café negro tradicional", TipoBebida.CALIENTE, {"cafe": 20}),
        ("Café con Leche", 3.00, "Café con leche cremosa", TipoBebida.CALIENTE, {"cafe": 20, "leche": 100}),
        ("Cappuccino", 3.50, "Café con leche espumosa", TipoBebida.CALIENTE, {"cafe": 20, "leche": 150}),
        ("Latte", 3.50, "Café con extra leche cremosa", TipoBebida.CALIENTE, {"cafe": 20, "leche": 200}),
        ("Mocha", 4.00, "Café con chocolate y leche", TipoBebida.CALIENTE, {"cafe": 20, "leche": 150, "chocolate": 30}),
        ("Té Negro", 2.50, "Té negro tradicional", TipoBebida.CALIENTE, {"te_negro": 10}),
        ("Té Verde", 2.50, "Té verde tradicional", TipoBebida.CALIENTE, {"te_verde": 10}),
        ("Frappuccino", 4.50, "Café helado con crema", TipoBebida.FRIA, {"cafe": 20, "leche": 150, "crema": 50}),
    ]

    for nombre, precio, desc, tipo, ingredientes in bebidas:
        bebida = Bebida(nombre, precio, desc, tipo)
        for ing, cant in ingredientes.items():
            bebida.agregar_ingrediente_base(ing, cant)
        cafe.agregar_al_menu(bebida)

    # Crear postres
    postres = [
        ("Croissant", 2.50, "Croissant de mantequilla", False, False, {"harina": 100, "huevos": 1}),
        ("Muffin de Arándanos", 3.00, "Muffin casero con arándanos", False, False, {"harina": 80, "huevos": 1, "frutas": 30}),
        ("Galleta Vegana", 2.50, "Galleta sin productos animales", True, False, {"harina": 50}),
        ("Brownie Sin Gluten", 3.50, "Brownie de chocolate sin gluten", False, True, {"chocolate": 50}),
    ]

    for nombre, precio, desc, vegano, sin_gluten, ingredientes in postres:
        postre = Postre(nombre, precio, desc, vegano, sin_gluten)
        postre.ingredientes_base = ingredientes
        cafe.agregar_al_menu(postre)

    # Crear promociones
    cafe.promociones = [
        Promocion("Descuento por Puntos", 10, puntos_requeridos=50),
        Promocion("Combo Desayuno", 15, productos_minimos=3),
    ]

    return cafe

def limpiar_pantalla():
    os.system('cls' if os.name == 'nt' else 'clear')

def menu_principal():
    print("\n=== SISTEMA DE CAFETERÍA ===")
    print("1. Realizar pedido")
    print("2. Consultar historial de cliente")
    print("3. Actualizar estado de pedido (empleados)")
    print("4. Ver inventario (empleados)")
    print("5. Ver menú")
    print("6. Salir")
    return input("Seleccione una opción: ")

def main():
    cafe = inicializar_cafeteria()

    # Crear algunos usuarios de ejemplo
    cliente1 = Cliente("Juan Pérez", "C001", "555-0123")
    cliente2 = Cliente("María García", "C002", "555-0124")
    barista = Empleado("Ana López", "E001", "555-0125", RolEmpleado.BARISTA)

    clientes = {"C001": cliente1, "C002": cliente2}
    empleados = {"E001": barista}

    while True:
        limpiar_pantalla()
        opcion = menu_principal()

        if opcion == "1":
            # Realizar pedido
            id_cliente = input("\nIngrese su ID de cliente (C001 o C002 para demo): ")
            if id_cliente not in clientes:
                print("Cliente no encontrado")
                input("Presione Enter para continuar...")
                continue

            cliente = clientes[id_cliente]
            cafe.mostrar_menu()

            productos_pedido = []
            while True:
                nombre_producto = input("\nIngrese el nombre del producto (o 'fin' para terminar): ")
                if nombre_producto.lower() == 'fin':
                    break

                if nombre_producto not in cafe.menu:
                    print("Producto no encontrado en el menú")
                    continue

                producto_base = cafe.menu[nombre_producto]
                producto_personalizado = ProductoPersonalizado(producto_base)

                # Preguntar por modificaciones
                while True:
                    modificacion = input("¿Desea agregar una modificación? (s/n): ")
                    if modificacion.lower() != 's':
                        break

                    print("\nModificaciones disponibles:")
                    print("- extra leche")
                    print("- extra chocolate")
                    print("- sin azúcar")
                    print("- leche de almendra")

                    mod = input("Ingrese la modificación: ")
                    if "extra" in mod:
                        ingrediente = mod.replace("extra ", "")
                        producto_personalizado.agregar_modificacion(ingrediente, 50)
                    elif "sin" in mod:
                        ingrediente = mod.replace("sin ", "")
                        producto_personalizado.agregar_modificacion(ingrediente, -1)

                productos_pedido.append(producto_personalizado)

            try:
                pedido = cliente.realizar_pedido(productos_pedido, cafe)
                print("\n¡Pedido realizado con éxito!")
                print(f"Total a pagar: ${pedido.total:.2f}")

                # Preguntar por promociones
                if cafe.promociones:
                    aplicar_promo = input("\n¿Desea aplicar promociones disponibles? (s/n): ")
                    if aplicar_promo.lower() == 's':
                        for promo in cafe.promociones:
                            if promo.es_aplicable(pedido):
                                pedido.aplicar_promocion(promo)
                                print(f"Promoción aplicada: {promo.nombre}")
                                print(f"Nuevo total: ${pedido.total:.2f}")

            except ValueError as e:
                print(f"\nError al realizar el pedido: {e}")

            input("\nPresione Enter para continuar...")

        elif opcion == "2":
            # Consultar historial
            id_cliente = input("\nIngrese su ID de cliente: ")
            if id_cliente in clientes:
                clientes[id_cliente].consultar_historial()
            else:
                print("Cliente no encontrado")
            input("\nPresione Enter para continuar...")

        elif opcion == "3":
            # Actualizar estado de pedido
            id_empleado = input("\nIngrese su ID de empleado: ")
            if id_empleado not in empleados:
                print("Empleado no autorizado")
                input("Presione Enter para continuar...")
                continue

            empleado = empleados[id_empleado]

            if not cafe.pedidos:
                print("No hay pedidos pendientes")
                input("Presione Enter para continuar...")
                continue

            print("\nPedidos actuales:")
            for i, pedido in enumerate(cafe.pedidos):
                print(f"\n{i+1}. Cliente: {pedido.cliente.nombre}")
                print(f"   Estado: {pedido.estado.value}")
                print(f"   Productos:")
                for producto in pedido.productos:
                    print(f"   - {producto.nombre}")

            try:
                num_pedido = int(input("\nSeleccione el número de pedido a actualizar: ")) - 1
                pedido = cafe.pedidos[num_pedido]

                print("\nEstados disponibles:")
                print("1. En preparación")
                print("2. Entregado")
                nuevo_estado = input("Seleccione el nuevo estado: ")

                if nuevo_estado == "1":
                    empleado.actualizar_estado_pedido(pedido, EstadoPedido.EN_PREPARACION)
                elif nuevo_estado == "2":
                    empleado.actualizar_estado_pedido(pedido, EstadoPedido.ENTREGADO)

            except (IndexError, ValueError):
                print("Selección inválida")

            input("Presione Enter para continuar...")

        elif opcion == "4":
            # Ver inventario
            id_empleado = input("\nIngrese su ID de empleado: ")
            if id_empleado in empleados:
                cafe.inventario.mostrar_stock()
            else:
                print("Empleado no autorizado")
            input("\nPresione Enter para continuar...")

        elif opcion == "5":
            # Ver menú
            cafe.mostrar_menu()
            input("\nPresione Enter para continuar...")

        elif opcion == "6":
            print("\n¡Gracias por usar el sistema de cafetería!")
            break

        else:
            print("Opción no válida")
            input("Presione Enter para continuar...")

if __name__ == "__main__":
    main()